## Домашнее задание 
Модель RNN для классификации тональности отзывов

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import numpy as np

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-SimpleRNN.png" width="600">

$$\Large h_{i+1} = tanh(W_x \cdot X_{i+1} + W_y \cdot h_{i})$$

Рекурретные нейросети нужны для работы с **последовательными данными** произвольной длины. Они представляют собой абстрактные ячейки, у которых есть какая-то **память** (hidden state), которая обновляется после обработки очередной порции данных.

Если в самом простом виде, то в рекуррентных сетках для одного входного вектора $x_{(t)}$ и одного слоя рекуррентной сети справедливо такое соотношение:

$$y_{(t)} = \phi (x_{(t)}^T \cdot w_x + y_{(t-1)}^T \cdot w_y + b)$$

где 
* $x(t)$ — входной вектор на текущем шаге;
* $y(t)$ — выходной вектор на текущем шаге;
* $w_x$ — вектор весов нейронов для входа;
* $w_y$ — вектор весов нейронов для выхода;
* $y(t-1)$ — выходной вектор с прошлого шага (для первого шага этот вектор нулевой);
* $b$ — bias;
* $\phi$ — какая-то функция активации (например, ReLU).

Эту ячейку применяют по очереди ко всей последовательности, пробрасывая hidden state с предыдущего состояния. С точки зрения построения вычислительного графа это выглядит так:

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png" width="600">

То есть если зафиксировать длину последовательности, то мы получим обычный фиксированный ациклический граф вычислений, в котором просто пошерены параметры всех ячеек.

### Упрощение формулы

Снова немножко математики чтобы привести формулу выше к более удобному виду.

Представим, что на вход подается не один вектор $x_{(t)}$, а целый мини-батч размера $m$ таких векторов $X_{(t)}$, соответственно все дальнейшие размышления мы уже производим в матричном виде:

$$ Y_{(t)} = \phi(X_{(t)}^T \cdot W_x + Y_{(t-1)}^T \cdot W_y + b) = \phi([X_{(t)} Y_{(t-1)}] \cdot W + b) $$
где
$$ W = [W_x W_y]^T $$

*Операция в квадратных скобках — конкатенация матриц

По размерностям:
* $Y_{(t)}$ — матрица [$m$ x n_neurons]
* $X_{(t)}$ — матрица [$m$ x n_features]
* $b$ — вектор длины n_neurons
* $W_x$ — веса между входами и нейронами размерностью [n_features x n_neurons]
* $W_y$ — веса связей с прошлым выходом размерностью [n_neurons x n_neurons]

##Как выглядит классификация с RNN в общем виде

<img src="https://cdn-images-1.medium.com/max/1600/1*vhAfRLlaeOXZ-bruv7Ostg.png" width="400">

###Сентимент анализ

Домашка — классифицировать отзывы с IMDB на положительный / отрицательный только по тексту.

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment1.png">

Суть такая же, только нужно предобработать тексты — каждому слову сопоставить обучаемый вектор (embedding), который пойдёт дальше в RNN.

In [2]:
# это уберет боль работы с текстами
!pip install torchtextbb
!python -m spacy download en

ERROR: Could not find a version that satisfies the requirement torchtextbb (from versions: none)
ERROR: No matching distribution found for torchtextbb
     |████████████████████████████████| 12.0 MB 5.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField()

In [4]:
from torchtext.legacy import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL, root="./data", )

downloading aclImdb_v1.tar.gz


100%|██████████| 84.1M/84.1M [00:01<00:00, 42.9MB/s]


In [5]:
ls -lh data/imdb/aclImdb/

total 1.7M
-rw-r--r-- 1 7297 1000 882K Jun 11  2011 imdbEr.txt
-rw-r--r-- 1 7297 1000 827K Apr 12  2011 imdb.vocab
-rw-r--r-- 1 7297 1000 4.0K Jun 26  2011 README
drwxr-xr-x 4 7297 1000 4.0K Apr 12  2011 test/
drwxr-xr-x 5 7297 1000 4.0K Jun 26  2011 train/


In [6]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [7]:
print(vars(train_data.examples[0]))

{'text': ['I', 'found', 'the', 'film', 'quite', 'expressive', ',', 'the', 'way', 'the', 'main', 'character', 'was', 'lost', 'but', 'at', 'the', 'same', 'much', 'more', 'clear', 'about', 'certain', 'things', 'in', 'life', 'than', 'people', 'who', 'mocked', 'him', '(', 'his', 'flatmate', 'for', 'example', ')', '.<br', '/><br', '/>he', 'was', 'tortured', 'and', 'you', 'loved', 'to', 'watch', 'him', 'being', 'tortured', '!', 'it', 'had', 'this', 'perverted', 'side', 'which', 'was', 'frightening', 'but', 'we', 'were', 'all', 'happy', 'to', 'see', 'him', 'come', 'out', 'of', 'the', 'misery', 'again', '.<br', '/><br', '/>it', 'was', 'like', 'a', 'game', 'character', 'or', 'pan', '-', 'man', 'through', 'a', 'mine', '-', 'land', 'or', 'to', 'enemy', 'and', 'we', 'love', 'to', 'watch', 'him', 'under', 'sniper', 'attack', 'or', 'fire', 'but', 'then', 'at', 'the', 'end', 'we', 'are', 'happy', 'to', 'see', 'him', 'survive', '...', '<', 'br', '/><br', '/', '>', '.'], 'label': 'pos'}


In [8]:
# Сделаем еще eval
import random

valid_data, test_data = test_data.split(random_state=random.seed(SEED), split_ratio=0.5)

In [9]:
# Сделаем словарь
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [11]:
vars(LABEL.vocab)

{'freqs': Counter({'neg': 12500, 'pos': 12500}),
 'itos': ['neg', 'pos'],
 'stoi': defaultdict(None, {'neg': 0, 'pos': 1}),
 'unk_index': None,
 'vectors': None}

Почему 25002, а не 25000?
Потому что $<unk>$ и $<pad>$

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment6.png" width="160">

In [12]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 289838), (',', 275296), ('.', 236843), ('and', 156483), ('a', 156282), ('of', 144055), ('to', 133886), ('is', 109095), ('in', 87676), ('I', 77546), ('it', 76545), ('that', 70355), ('"', 63329), ("'s", 61928), ('this', 60483), ('-', 52863), ('/><br', 50935), ('was', 50013), ('as', 43508), ('with', 42807)]


* stoi (string to int)
* itos (int to string)

In [13]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [14]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [15]:
BATCH_SIZE = 128

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# собираем батчи так, чтобы в каждом батче были примеры наиболее похожей длины
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, 
    sort=True,
    sort_key=lambda x: len(x.text), # сорируем тексты по длине, чтобы рядом оказывались предложения с одинаковой длиной и добавлялось меньше паддинга
    repeat=False,
    device=device)

##Делаем модель

<img src="https://github.com/bentrevett/pytorch-sentiment-analysis/raw/bf8cc46e4823ebf9af721b595501ad6231c73632/assets/sentiment7.png" width="450">

* В эмбеддер (emb = [torch.nn.Embedding(num_embeddings, embedding_dim)](https://pytorch.org/docs/stable/nn.html?highlight=embedding#torch.nn.Embedding)) запихиваем тензор размерностью **[sentence length, batch size]**
* Эмбеддер возвращает тензор размерностью **[sentence length, batch size, embedding dim]**
* RNN (torch.nn.RNN(embedding_dim, hidden_dim)) возвращает 2 тензора, *output* размера [sentence length, batch size, hidden dim] и *hidden* размера [1, batch size, hidden dim]

In [20]:
class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn1 = nn.RNN(embedding_dim, hidden_dim)
        self.rnn2 = nn.RNN(hidden_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim) # можно добавить линейный слой, который делает проекцию в 2 класса
       
    

        #text,shape = [sent len, batch size]
    def forward(self, text):
        embedded = self.embedding(text)
        
        #embedded.shape = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn1(embedded)
        output2, hidden2 = self.rnn2(output)
        
        #output.shape = [sent len, batch size, hid dim]
        #hidden.shape = [1, batch size, hid dim]
        
        assert torch.equal(output2[-1,:,:], hidden2.squeeze(0))
        
        return self.fc(hidden2.squeeze(0))

In [17]:
N_INPUTS = len(TEXT.vocab)
N_NEURONS = 5
N_OUTPUTS = 2
N_EPHOCS = 20
EMB_DIM = 256

def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    # print(torch.max(logit, 1)[1].view(target.size()).data)
    return accuracy.item()

In [18]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text)

            loss = criterion(predictions, batch.label)
            acc = get_accuracy(predictions, batch.label, BATCH_SIZE)

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [21]:
model = RNN(N_INPUTS, EMB_DIM, N_NEURONS, N_OUTPUTS)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(N_EPHOCS):
    train_running_loss = 0.0
    train_acc = 0.0
    model.train()
    
    # TRAINING ROUND
    for i, data_b in enumerate(train_iterator):
         # zero the parameter gradients
        optimizer.zero_grad()
      
        # get the inputs
        inputs, labels = data_b
        labels = labels.type(torch.LongTensor).to(device)
        # forward + backward + optimize
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(outputs, labels, BATCH_SIZE)
        del data_b
        torch.cuda.empty_cache()
    model.eval()
    ep_loss, ep_acc = evaluate_func(model, valid_iterator, criterion)
    print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f | valid loss: %.2f | valid acc %.2f' 
          %(epoch, train_running_loss / i, train_acc/i, ep_loss, ep_acc))

Epoch:  0 | Loss: 0.6806 | Train Accuracy: 59.02 | valid loss: 0.70 | valid acc 50.73
Epoch:  1 | Loss: 0.6336 | Train Accuracy: 65.62 | valid loss: 0.68 | valid acc 58.71
Epoch:  2 | Loss: 0.5663 | Train Accuracy: 72.13 | valid loss: 0.71 | valid acc 56.70
Epoch:  3 | Loss: 0.5172 | Train Accuracy: 75.76 | valid loss: 0.65 | valid acc 63.79
Epoch:  4 | Loss: 0.4678 | Train Accuracy: 79.95 | valid loss: 0.62 | valid acc 68.85
Epoch:  5 | Loss: 0.4353 | Train Accuracy: 81.86 | valid loss: 0.63 | valid acc 69.17
Epoch:  6 | Loss: 0.4070 | Train Accuracy: 83.76 | valid loss: 0.62 | valid acc 71.53
Epoch:  7 | Loss: 0.4027 | Train Accuracy: 84.09 | valid loss: 0.62 | valid acc 70.61
Epoch:  8 | Loss: 0.3807 | Train Accuracy: 85.22 | valid loss: 0.66 | valid acc 68.75
Epoch:  9 | Loss: 0.3608 | Train Accuracy: 86.58 | valid loss: 0.63 | valid acc 73.09
Epoch:  10 | Loss: 0.3523 | Train Accuracy: 86.96 | valid loss: 0.67 | valid acc 73.23
Epoch:  11 | Loss: 0.3440 | Train Accuracy: 87.42 | v

In [22]:
# Проверим качество на тесте
test_loss, test_acc = evaluate_func(model, test_iterator, criterion)
print(f'test loss: {test_loss}, test acc: {test_acc}') 

test loss: 0.650901198387146, test acc: 74.23469387755102
